# KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [3]:
import pandas as pd

#Como visto anteriormente, as funcoes basicas do pandas sobre ler um arquivo csv e transformar em tabela
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


user_id  movie_id  rating
0        0        50       5
1        0       172       5
2        0       133       1
3      196       242       3
4      186       302       3

Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [13]:
#Quando eu rodei o codigo, apareceu um aviso, que ocorre porque, em versões futuras do pandas, a forma como o método .agg() 
#lida com funções como np.mean será alterada, dito isso, o pandas recomenda usar o nome do método como 
#uma string ('mean' em vez de np.mean) para garantir compatibilidade futura.
movieProperties = ratings.groupby('movie_id').agg({'rating': ['size', 'mean']})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [5]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
#é usado esta funcao lambada para poder efetuar a comparacao em realacao ao maximo e minimo numero de avaliacoes
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
#da uma escala de popularidade em uma escala de 0 a 1
movieNormalizedNumRatings.head()

size
movie_id          
1         0.773585
2         0.222985
3         0.152659
4         0.356775
5         0.145798

Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [15]:
#desta vez, em vez de usar pandas para abrir o arquivo, foi aberto de uma maneira apenas com o python
movieDict = {}
with open(r'u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        #sendo iterada uma linha por vez ate acabar o arquivo
        fields = line.rstrip('\n').split('|')
        #extraindo o movieID, name, genres diferentes
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


For example, here's the record we end up with for movie ID 1, "Toy Story":

In [ ]:
#O array demonstra os 0 para os generos que nao sao compativeis e o 1 para os que sao, apos isso tem a area
#da classificacao de uma escala de 0 a 1, sendo 1 o mais alto, e o ultimo a media para ToyStory

In [16]:
print(movieDict[1])

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.7735849056603774, 3.8783185840707963)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [17]:
#usada para calcular a distância do cosseno entre vetores.
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
#Calcula a distância do cosseno entre os vetores de gênero dos dois filmes,
#usando essa métrica avalia a similaridade entre os vetores, mais proximo de 0, mais parecido
    genreDistance = spatial.distance.cosine(genresA, genresB)
#Calcula a diferenca da popularidade absoluta entre os filmes
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
#retorna a soma dos dois a soma da distancia dos generos com a diferenca do popularidade dos filmes
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[2], movieDict[4])



0.8004574042309892

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [18]:
print(movieDict[2])
print(movieDict[4])


('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [19]:
import operator

#calcula a distância em relação ao filme alvo 
def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    #Ordena os filmes pela distância, em ordem crescente.
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    #pega os K filmes que se mais se parecem e ordena pelo id
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
#Para cada vizinho, soma a avaliação armazenada no índice
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
#Divide a soma das avaliações pelo número de vizinhos    
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [20]:
avgRating

3.3445905900235564

How does this compare to Toy Story's actual average rating?

In [21]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

Not too bad!


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

In [26]:
import operator

# Calcula a distância entre o filme alvo e os demais
def ComputeDistance(movieA, movieB):
    # Ajusta a metrica para priorizar a diferenca de popularidade
    popularity_diff = abs(movieA[2] - movieB[2])  # Diferenca de popularidade
    genre_similarity = sum([a * b for a, b in zip(movieA[1], movieB[1])])  # Generos similares
    return popularity_diff - genre_similarity  # Maior diferença de popularidade menos a distância

def getNeighbors(movieID, K, distance_metric):
    distances = []
    for movie in movieDict:
        if movie != movieID:
            dist = distance_metric(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    # Ordena os filmes pela distância, em ordem crescente (menor distância é mais próximo)
    distances.sort(key=operator.itemgetter(1))
    neighbors = [distances[x][0] for x in range(K)]
    return neighbors

K = 10
# Chama a função getNeighbors para encontrar os K vizinhos mais próximos
neighbors = getNeighbors(1, K, improvedDistance)
# Calcula a média das avaliações
avgRating = sum([movieDict[neighbor][3] for neighbor in neighbors]) / K
#Imprime o nome  e a avaliação dos vizinhos
for neighbor in neighbors:
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
print(f"Average Rating with improved metric: {avgRating:.2f}")

Aladdin (1992) 3.8127853881278537
Space Jam (1996) 2.774193548387097
Hercules (1997) 3.515151515151515
Aladdin and the King of Thieves (1996) 2.8461538461538463
Goofy Movie, A (1995) 2.9
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Lion King, The (1994) 3.7818181818181817
Babe (1995) 3.9954337899543377
Beauty and the Beast (1991) 3.792079207920792
Mary Poppins (1964) 3.7247191011235956
Average Rating with improved metric: 3.48


In [27]:
avgRating

3.4774236419128015

In [29]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)